# 공영주차장 중 우선설치 입지 선정 작업

### 공영주차장/ 충전소 매칭 장소 시각화 작업

In [22]:
import pandas as pd, geopandas as gpd, plotly.express as px
import plotly.io as pio

px.set_mapbox_access_token("YOUR_MAPBOX_ACCESS_TOKEN")

# plotly+Mapbox, browser 환경에 오픈
pio.renderers.default = 'browser'

# CSV 읽기
ch = pd.read_csv("수도권_충전소데이터_민공분리.csv")
pk = pd.read_csv("수도권_주차장데이터_중복값제거.csv")
pk_public = pk[pk['주차장구분'].str.strip() == '공영']

# ch와 pk_public을 GeoDataFrame으로 변환
# gpd.sjoin_nearest를 사용하기 위함
gdf_ch = gpd.GeoDataFrame(ch,
    geometry=gpd.points_from_xy(ch['경도'], ch['위도']), crs="EPSG:4326")
gdf_pk = gpd.GeoDataFrame(pk_public,
    geometry=gpd.points_from_xy(pk_public['경도'], pk_public['위도']), crs="EPSG:4326")

# 투영 / 근접 조인
# EPSG:5179로 sjoin_nearest에서 max_distance를 설정해야 m를 기준으로 거리를 설정할 수 있다.
proj = "EPSG:5179"
gdf_ch = gdf_ch.to_crs(proj)
gdf_pk = gdf_pk.to_crs(proj)

matched = gpd.sjoin_nearest(
    gdf_ch, gdf_pk[['주차장관리번호','geometry']],
    how="inner", max_distance=50
).to_crs("EPSG:4326")

# Plotly Mapbox 시각화
fig = px.scatter_mapbox(
    matched,
    lat=matched.geometry.y,
    lon=matched.geometry.x,
    hover_name="충전소명",
    size_max=2,  # 최대 크기 제한 (기본은 20)
    size=[5]*len(matched),  # 모든 점을 5 크기로 고정
    color_discrete_sequence=["blue"],
    zoom=8,
    mapbox_style="carto-positron",   # 토큰 필요 스타일
    title="공영주차장 내 전기차 충전소"
)
fig.show()


In [31]:
import pandas as pd, plotly.express as px, plotly.io as pio

# Mapbox 토큰 및 렌더러 설정
px.set_mapbox_access_token("YOUR_MAPBOX_ACCESS_TOKEN")
pio.renderers.default = 'browser'

# CSV 읽기 및 컬럼 공백 제거
df = pd.read_csv("30년도_수도권_이동_필터링.csv")
df.columns = df.columns.str.strip()

# density heatmap 생성
fig = px.density_mapbox(
    df,
    lat="lat",               # 위도 컬럼
    lon="lon",               # 경도 컬럼
    z="value",             # 밀도 기준값
    radius=5,               # 히트맵 반경(픽셀)
    center={"lat":37.5,"lon":127},  # 초기 중심 좌표
    zoom=8,                  # 초기 줌 레벨
    mapbox_style="carto-positron",
    color_continuous_scale="Blues",
    title="수도권 EV 이동량 (2030)"
)

# 브라우저에서 시각화
fig.show()


In [47]:
import pandas as pd

# 가장 유연하게 감지하는 방식
df = pd.read_csv("수도권_주차장데이터_UTF8.csv", encoding="cp949")
df.to_csv("수도권_주차장데이터.csv", encoding="utf-8-sig", index=False)

In [51]:
df = pd.read_csv("수도권_충전소데이터_UTF8.csv", encoding="cp949")
df.to_csv("수도권_충전소데이터.csv", index=False, encoding="utf-8-sig")

### 원래 의도는 공영 주차장 데이터와 민영/공영 분리 충전소 데이터를 겹쳐서 충전소가 설치된 공영주차장과
### 설치되지 않은 공영 주차장을 확인하려 했으나, 서울에선 모든 공영 주차장에 충전소가 설치되었고,
### 경기도나 인천광역시까지 확인하는건 무의미한것으로 판단되어 이 이상의 작업은 중단함.